In [2]:
!*rm -f go.work && go work init && go work use . "${HOME}/Projects/gomlx"
%goworkfix

	- Added replace rule for module "github.com/gomlx/gomlx" to local directory "/home/janpf/Projects/gomlx".


## Downloading Dataset

The method `mag.Download()` will download the dataset to the data directory, if not yet downloaded.
It then converts the dataset to tensors, which are then available for use.

The tensor is then saved for faster access. After saved, the next call to `mag.Download()` will take 1/2s.

In [3]:
%env GOMLX_PLATFORM=GPU

Set: GOMLX_PLATFORM="GPU"


In [4]:
import (
    "flag"
    mag "github.com/gomlx/gomlx/examples/ogbnmag"
    "github.com/janpfeifer/must"
)

var (
    flagDataDir   = flag.String("data", "~/work/ogbnmag", "Directory to cache downloaded and generated dataset files.")
)

%%
start := time.Now()
must.M(mag.Download(*flagDataDir))
magSampler := must.M1(mag.NewSampler(*flagDataDir))

fmt.Printf("Elapsed: %s\n", time.Since(start))
fmt.Printf("%s\n", magSampler.String())

Elapsed: 1.050133099s
Sampler: 4 node types, 8 edge types
	NodeType "fields_of_study": 59,965 items
	NodeType "papers": 736,389 items
	NodeType "authors": 1,134,649 items
	NodeType "institutions": 8,740 items
	EdgeType "cites": ["papers"]->["papers"], 5,416,271 edges
	EdgeType "citedBy": ["papers"]->["papers"], 5,416,271 edges
	EdgeType "affiliatedWith": ["authors"]->["institutions"], 1,043,998 edges
	EdgeType "affiliations": ["institutions"]->["authors"], 1,043,998 edges
	EdgeType "hasTopic": ["papers"]->["fields_of_study"], 7,505,078 edges
	EdgeType "topicHasPapers": ["fields_of_study"]->["papers"], 7,505,078 edges
	EdgeType "writes": ["authors"]->["papers"], 7,145,660 edges
	EdgeType "writtenBy": ["papers"]->["authors"], 7,145,660 edges


## FNN model, No Graph

The first model will only use the paper features, and no relations. It serves as a baseline.

In a quick experiment, without much hyperparameter tuning, we got 27.27% on test accuracy, which is inline with the corresponding results in [the leaderboard](https://ogb.stanford.edu/docs/leader_nodeprop/#ogbn-mag) (the Multi Layer Perceptron MLP entry in the bottom).

In [40]:
import (
	. "github.com/gomlx/gomlx/graph"
	"github.com/gomlx/gomlx/ml/context"
)
var manager = NewManager()

func config() (ctx *context.Context) {
    ctx = context.NewContext(manager)
    ctx.RngStateReset()
    ctx.SetParams(map[string]any{
        "train_steps": 2_000_000, 
        "batch_size": 128,
        "optimizer": "adamw", 
        optimizers.LearningRateKey: 0.0001,
        layers.L2RegularizationKey: 1e-4,
        "normalization": "layer",
        "dropout": 0.1,
        "hidden_layers": 2,
        "num_nodes": 256,
        "plots": true,
    })
    return
}

In [41]:
import "github.com/gomlx/gomlx/examples/ogbnmag/fnn"

%%
must.M(mag.Download(*flagDataDir))
ctx := config()
ctx.SetParam("checkpoint", path.Join(*flagDataDir, "fnn-baseline"))
ctx.SetParam("num_checkpoints", 10)
err := fnn.Train(ctx)
if  err != nil {
    fmt.Printf("%+v\n", err)
}


<path vector-effect="non-scaling-stroke" d="M1.768000e+01,6.106156e+00 L9.804988e+01,1.564710e+01 L1.473386e+02,4.211405e+01 L1.839269e+02,7.153961e+01 L2.135617e+02,1.098639e+02 L2.389345e+02,1.193361e+02 L2.613214e+02,1.045660e+02 L2.815843e+02,1.456425e+02 L3.002547e+02,1.383950e+02 L3.176798e+02,1.373630e+02 L3.340964e+02,1.463076e+02 L3.496716e+02,1.457113e+02 L3.645714e+02,1.441288e+02 L3.789094e+02,1.483947e+02 L3.927645e+02,1.471103e+02 L4.061928e+02,1.565366e+02 L4.192612e+02,1.511698e+02 L4.320140e+02,1.442434e+02 L4.445019e+02,1.565366e+02 L4.567564e+02,1.569953e+02 L4.688127e+02,1.588989e+02 L4.806910e+02,1.624768e+02 L4.924143e+02,1.703205e+02 L5.039938e+02,1.653436e+02 L5.154434e+02,1.680729e+02 L5.267777e+02,1.621557e+02 L5.380103e+02,1.732332e+02 L5.491530e+02,1.797238e+02 L5.602152e+02,1.733938e+02 L5.712041e+02,1.791963e+02 L5.821303e+02,1.799303e+02 L5.929999e+02,1.738066e+02 L6.038156e+02,1.829347e+02 L6.145861e+02,1.816963e+02 L6.253144e+02,1.802284e+02 L6.360033e+02,1.770863e+02 L6.466577e+02,1.907097e+02 L6.572800e+02,1.872695e+02 L6.678758e+02,1.899987e+02 L6.784469e+02,1.908244e+02 L6.889962e+02,1.965810e+02 L6.995246e+02,1.948150e+02 L7.100349e+02,1.968333e+02 L7.205283e+02,1.920629e+02 L7.310061e+02,2.012139e+02 L7.414703e+02,1.972232e+02 L7.519221e+02,2.047000e+02 L7.623623e+02,2.011222e+02 L7.727920e+02,2.089659e+02 L7.832123e+02,2.067183e+02 L7.936243e+02,2.053193e+02 L8.040282e+02,2.106172e+02 L8.144245e+02,2.116264e+02 L8.248145e+02,2.070394e+02 L8.351988e+02,2.073146e+02 L8.455774e+02,2.100668e+02 L8.559509e+02,2.040349e+02 L8.663200e+02,2.097687e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,1.656091e+01 L9.804988e+01,5.587929e+01 L1.473386e+02,7.048981e+01 L1.839269e+02,9.179369e+01 L2.135617e+02,9.879721e+01 L2.389345e+02,1.148011e+02 L2.613214e+02,1.212928e+02 L2.815843e+02,1.301297e+02 L3.002547e+02,1.341891e+02 L3.176798e+02,1.414829e+02 L3.340964e+02,1.474047e+02 L3.496716e+02,1.539529e+02 L3.645714e+02,1.578718e+02 L3.789094e+02,1.645866e+02 L3.927645e+02,1.676636e+02 L4.061928e+02,1.708766e+02 L4.192612e+02,1.757671e+02 L4.320140e+02,1.792154e+02 L4.445019e+02,1.836293e+02 L4.567564e+02,1.871264e+02 L4.688127e+02,1.889720e+02 L4.806910e+02,1.932835e+02 L4.924143e+02,1.952055e+02 L5.039938e+02,1.977585e+02 L5.154434e+02,1.992817e+02 L5.267777e+02,2.021479e+02 L5.380103e+02,2.051043e+02 L5.491530e+02,2.070003e+02 L5.602152e+02,2.074357e+02 L5.712041e+02,2.112232e+02 L5.821303e+02,2.120558e+02 L5.929999e+02,2.138403e+02 L6.038156e+02,2.156752e+02 L6.145861e+02,2.191067e+02 L6.253144e+02,2.204054e+02 L6.360033e+02,2.213220e+02 L6.466577e+02,2.231478e+02 L6.572800e+02,2.254563e+02 L6.678758e+02,2.263211e+02 L6.784469e+02,2.274853e+02 L6.889962e+02,2.308327e+02 L6.995246e+02,2.316394e+02 L7.100349e+02,2.327089e+02 L7.205283e+02,2.339311e+02 L7.310061e+02,2.363909e+02 L7.414703e+02,2.378744e+02 L7.519221e+02,2.395795e+02 L7.623623e+02,2.392785e+02 L7.727920e+02,2.420270e+02 L7.832123e+02,2.433272e+02 L7.936243e+02,2.445112e+02 L8.040282e+02,2.448856e+02 L8.144245e+02,2.467617e+02 L8.248145e+02,2.480237e+02 L8.351988e+02,2.492078e+02 L8.455774e+02,2.498082e+02 L8.559509e+02,2.511863e+02 L8.663200e+02,2.524529e+02 "/> <path vector-effect="non-scaling-stroke" d="M1.768000e+01,5.200000e+00 L9.804988e+01,2.346513e+01 L1.473386e+02,3.115961e+01 L1.839269e+02,4.681543e+01 L2.135617e+02,7.751923e+01 L2.389345e+02,8.900907e+01 L2.613214e+02,8.321226e+01 L2.815843e+02,1.209137e+02 L3.002547e+02,1.196536e+02 L3.176798e+02,1.188678e+02 L3.340964e+02,1.293643e+02 L3.496716e+02,1.345829e+02 L3.645714e+02,1.298536e+02 L3.789094e+02,1.370588e+02 L3.927645e+02,1.389417e+02 L4.061928e+02,1.473626e+02 L4.192612e+02,1.450053e+02 L4.320140e+02,1.444568e+02 L4.445019e+02,1.535449e+02 L4.567564e+02,1.541972e+02 L4.688127e+02,1.575774e+02 L4.806910e+02,1.608242e+02 L4.924143e+02,1.692007e+02 L5.039938e+02,1.653312e+02 L5.154434e+02,1.697789e+02 L5.267777e+02,1.648123e+02 L5.380103e+02,1.752050e+02

> papers features input shape: (Float32)[128 132]
Training (2000000 steps):   13% [====>...................................] (2069 steps/s) 19s:13m58s] [loss=3.070] [~loss=2.987] [~acc=29.03%]        

^C
signal: interrupt


## GNN (Graph Neural Networks)

To use GNNs we need first graphs to run the model on. The issue is, generally the graphs in real life are too large (social networks, relational datases, etc.), including OGNB-MAG.

So instead we use sampled sub-graphs to train. For inference we can also use sampled subgraphs, but later we show a work around where we can do inference, a node type at a time, where we don't need sampling.

### Sampling Sub-Graphs

We follow the same sampling strategy used in [TensorFlow GNN](https://github.com/tensorflow/gnn) library, describe it its [OGBN-MAG notebook](https://github.com/tensorflow/gnn/blob/main/examples/notebooks/ogbn_mag_e2e.ipynb).

In [6]:
import "github.com/gomlx/gomlx/examples/ogbnmag/sampler"

func GnnStrategy(magSampler *sampler.Sampler, batchSize int, seedIdsCandidates []int32) *sampler.Strategy {
    strategy := magSampler.NewStrategy()
    var seeds *sampler.Rule
    if seedIdsCandidates == nil {
        seeds = strategy.Nodes("seeds", "papers", batchSize)
    } else {
        seeds = strategy.NodesFromSet("seeds", "papers", batchSize, seedIdsCandidates)
    }    
    citations := seeds.FromEdges("citations", "cites", 8)

    // Authors
    seedsAuthors := seeds.FromEdges("seedsAuthors", "writtenBy", 8)
    citationsAuthors := citations.FromEdges("citationsAuthors", "writtenBy", 8)

    // Co-authored papers
    coauthoredPapers := seedsAuthors.FromEdges("coauthoredPapers", "writes", 8)
    coauthoredFromCitations := citationsAuthors.FromEdges("coauthoredFromCitations", "writes", 8)
    
     // Affiliations
    _ = seedsAuthors.FromEdges("authorsInstitutions", "affiliatedWith", 8)
    _ = citationsAuthors.FromEdges("citationAuthorsInstitutions", "affiliatedWith", 8)

    // Topics
    _ = seeds.FromEdges("seedsTopics", "hasTopic", 8)
    _ = coauthoredPapers.FromEdges("coauthoredTopics", "hasTopic", 8)
    _ = citations.FromEdges("citationsTopics", "hasTopic", 8)
    _ = coauthoredFromCitations.FromEdges("coauthoredFromCitationsTopics", "hasTopic", 8)
    
    return strategy
}

%%
must.M(mag.Download(*flagDataDir))
magSampler := must.M1(mag.NewSampler(*flagDataDir))
strategy := GnnStrategy(magSampler, 32, mag.TrainSplit.Local().FlatCopy().([]int32))
fmt.Printf("%s\n", strategy)


Sampling strategy: (12 rules)
> Rule "seeds": type=Node, nodeType="papers", shape=(Int32)[32] (size=32), nodeSet.size=629571
  > Rule "citations": type=Edge, nodeType="papers", shape=(Int32)[32 8] (size=256), sourceRule="seeds", edgeType="cites"
    > Rule "citationsAuthors": type=Edge, nodeType="authors", shape=(Int32)[32 8 8] (size=2048), sourceRule="citations", edgeType="writtenBy"
      > Rule "coauthoredFromCitations": type=Edge, nodeType="papers", shape=(Int32)[32 8 8 8] (size=16384), sourceRule="citationsAuthors", edgeType="writes"
        > Rule "coauthoredFromCitationsTopics": type=Edge, nodeType="fields_of_study", shape=(Int32)[32 8 8 8 8] (size=131072), sourceRule="coauthoredFromCitations", edgeType="hasTopic"
      > Rule "citationAuthorsInstitutions": type=Edge, nodeType="institutions", shape=(Int32)[32 8 8 8] (size=16384), sourceRule="citationsAuthors", edgeType="affiliatedWith"
    > Rule "citationsTopics": type=Edge, nodeType="fields_of_study", shape=(Int32)[32 8 8] (si

### Benchmarking Sampling

Let's get some benchmarking on the sampler, to make sure it's not slowing training:

1. Benchmark sequential the sampling of one sub-graph: `BenchmarkSequentialMagSampler`
2. Benchmark parallel sampling of one sub-graph: `BenchmarkParallelMagSampler`
3. Benchmark parallel sampling of one epoch (using parallel sampler): `BenchmarkOneEpochMagSampler`

In [38]:
import (
    mldata "github.com/gomlx/gomlx/ml/data"
)

func BenchmarkSequentialMagSampler(b *testing.B) {
    must.M(mag.Download(*flagDataDir))
    magSampler := must.M1(mag.NewSampler(*flagDataDir))
    strategy := GnnStrategy(magSampler, 32, mag.TrainSplit.Local().FlatCopy().([]int32))
    
    ds := strategy.NewDataset("train").Infinite().Shuffle()
    b.ResetTimer()
    for _ = range b.N {
        _, inputs, _, err := ds.Yield()
        if err != nil {
            b.Fatalf("Failed to sample: %+v", err)
        }
        _ = inputs
    }
}

func BenchmarkParallelMagSampler(b *testing.B) {
    must.M(mag.Download(*flagDataDir))
    magSampler := must.M1(mag.NewSampler(*flagDataDir))
    strategy := GnnStrategy(magSampler, 32, mag.TrainSplit.Local().FlatCopy().([]int32))
    
    ds := strategy.NewDataset("train").Infinite().Shuffle()
    parallelDS := mldata.Parallel(ds)
    b.ResetTimer()
    for _ = range b.N {
        _, inputs, _, err := parallelDS.Yield()
        if err != nil {
            b.Fatalf("Failed to sample: %+v", err)
        }
        _ = inputs
    }
}

func BenchmarkOneEpochMagSampler(b *testing.B) {
    must.M(mag.Download(*flagDataDir))
    magSampler := must.M1(mag.NewSampler(*flagDataDir))
    strategy := GnnStrategy(magSampler, 32, mag.TrainSplit.Local().FlatCopy().([]int32))
    
    ds := strategy.NewDataset("train").Shuffle()
    parallelDS := mldata.Parallel(ds)
    b.ResetTimer()
    var count int
    for _ = range b.N {
        count = 0
        for {
            _, inputs, _, err := parallelDS.Yield()
            if err == io.EOF {
                break
            }
            if err != nil {
                b.Fatalf("Failed to sample: %+v", err)
            }
            _ = inputs
            count ++
        }
        parallelDS.Reset()
    }
}

%%
%test -test.bench=. -test.run=Benchmark -test.benchtime=5s

goos: linux
goarch: amd64
pkg: gonb_e851bff2
cpu: 12th Gen Intel(R) Core(TM) i9-12900K
BenchmarkOneEpochMagSampler-24      	       3	1829138955 ns/op
BenchmarkParallelMagSampler-24      	   69396	     89149 ns/op
BenchmarkSequentialMagSampler-24    	   12021	    520495 ns/op
PASS
